In [1]:
data = [line.strip().split('\t') for line in open('project3_dataset1.txt', 'r')]

In [2]:
import numpy as np
import math
data = np.asarray(data)
print(data)
classlabels = data[:,-1].reshape((data.shape[0],1)).astype(int)
data = data[:,0:-1]
print(classlabels)
print(data)

[['20.13' '28.25' '131.2' ... '0.2572' '0.06637' '1']
 ['12.45' '15.7' '82.57' ... '0.3985' '0.1244' '1']
 ['11.26' '19.96' '73.72' ... '0.2955' '0.07009' '0']
 ...
 ['14.5' '10.89' '94.28' ... '0.2889' '0.08006' '0']
 ['12.36' '18.54' '79.01' ... '0.2983' '0.07185' '0']
 ['11.93' '21.53' '76.53' ... '0.2438' '0.08541' '0']]
[[1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0

In [3]:
def is_categorical(s):
    try:
        float(s)
    except ValueError:
        return True
    return False


def evaluate(actual, predicted):
    # a=TP, b= FN, c=FP, d= TN
    a = b = c = d = 0
    for i in range(len(actual)):
        if(actual[i] == 1 and predicted[i] == 1):
            a += 1
        elif(actual[i] == 1 and predicted[i] == 0):
            b += 1
        elif(actual[i] == 0 and predicted[i] == 1):
            c += 1
        elif(actual[i] == 0 and predicted[i] == 0):
            d += 1
    return a, b, c, d

In [4]:
strindex = []
for i in range(len(data[0])):
    if(is_categorical(data[0][i])):
        print(data[0][i])
        print(i)
        strindex.append(i)
for i in strindex:
    unique_vals = np.unique(data[:,i])
    mapping = list(range(len(unique_vals)))
    dictionary = dict(zip(unique_vals, mapping))
    for j in range(len(data[:,i])):
        data[j][i] = dictionary.get(data[j][i])

data = data.astype(float)
data = np.append(data,classlabels,axis=1)
print(data)

[[2.013e+01 2.825e+01 1.312e+02 ... 2.572e-01 6.637e-02 1.000e+00]
 [1.245e+01 1.570e+01 8.257e+01 ... 3.985e-01 1.244e-01 1.000e+00]
 [1.126e+01 1.996e+01 7.372e+01 ... 2.955e-01 7.009e-02 0.000e+00]
 ...
 [1.450e+01 1.089e+01 9.428e+01 ... 2.889e-01 8.006e-02 0.000e+00]
 [1.236e+01 1.854e+01 7.901e+01 ... 2.983e-01 7.185e-02 0.000e+00]
 [1.193e+01 2.153e+01 7.653e+01 ... 2.438e-01 8.541e-02 0.000e+00]]


In [5]:
# print(data[:,4])
print(data.shape)

(569, 31)


In [6]:
def calculate_gini(left, right):
    prob_left0 = prob_left1 = prob_right0 = prob_right1 = 0
    l = len(left)
    r = len(right)
    total = l + r
    if(l != 0):
        prob_left0 = float(list(left[:,-1]).count(0)) / l
        prob_left1 = float(list(left[:,-1]).count(1)) / l
    if(r != 0):
        prob_right0 = float(list(right[:,-1]).count(0)) / r
        prob_right1 = float(list(right[:,-1]).count(1)) / r
    
    gini_left = 1.0 - ((prob_left0 ** 2) + (prob_left1 ** 2))
    gini_right = 1.0 - ((prob_right0 ** 2) + (prob_right1 ** 2))
    
    gini = float((gini_left * l) + (gini_right * r)) / total
    return gini


In [7]:
def split_data(data,j,splitval):
    left = []
    right = []
    for i in range(len(data)):
        if(data[i][j] < splitval):
            left.append(data[i])
        else:
            right.append(data[i])
    left = np.asarray(left)
    right = np.asarray(right)
    return left, right

In [8]:
def node_to_split(data):
    min = float('inf')
    node = dict()
    flag=0
    for j in range(len(data[0])-1):
        if(splitted[j]!=1):
            for i in range(len(data)):
                    left, right = split_data(data,j,data[i][j])
                    gini = calculate_gini(left, right)
                    if(gini < min):
                        min = gini
                        node['gini'] = min
                        node['split_id'] = i
                        node['splitval'] = data[i][j]
                        node['split_feat'] = j
                        node['left'] = left
                        node['right'] = right
                        splitj = j
                        flag=1
    if(flag==1):
        splitted[splitj] = 1                   
    return node


def node_label(left, right):
    num0s = 0
    num1s = 0
    if(len(left) != 0):
        num0s += list(left[:,-1]).count(0)
        num1s += list(left[:,-1]).count(1)
    if(len(right) != 0):
        num0s += list(right[:,-1]).count(0)
        num1s += list(right[:,-1]).count(1)
    return 1 if(num1s > num0s) else 0


def is_sameclass(data):
    unique_labels = np.unique(data[:,-1])
    if(len(unique_labels) == 1):
        return True
    return False


In [9]:
def build_tree(node):
    left = node['left']
    right = node['right']
    del(node['left'])
    del(node['right'])

    if(len(left) == 0 or len(right) == 0):
        node['left'] = node['right'] = node_label(left, right)
        return node 
    #     if((left.size==0) or (right.size==0)):
#         classes = list()
#         if(left.size==0):
#             classes = right[:,-1]  
#         else:
#             classes = left[:,-1]
            
#         node['left'] = node['right'] = max(np.unique(classes), key = list(classes).count)
#         return node

    if(len(left) > 0):
        if(is_sameclass(left)):
            node['left'] = node_label(left, [])
        else:
            node['left'] = build_tree(node_to_split(left))
    if(len(right) > 0):
        if(is_sameclass(right)):
            node['right'] = node_label([], right)
        else:
            node['right'] = build_tree(node_to_split(right))        
    return node


In [10]:
def classify(node, test_point):
    
    #     if(test_point[node['split_feat']] < node['splitval']):
#         if(node['left'] == 0 or node['left'] == 1):
#             return node['left']                                 
#         else:
#             return classify(node['left'], test_point)
#     else:
#         if(node['right'] == 0 or node['right'] == 1):
#             return node['right']
#         else:
#             return classify(node['right'], test_point)
    
    if(test_point[node['split_feat']] < node['splitval']):
        if isinstance(node['left'], dict):
            return classify(node['left'], test_point)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return classify(node['right'], test_point)
        else:
            return node['right']
    


In [11]:
def split(kfoldlist,index):
    test = kfoldlist[index]
    train = np.vstack([j for i,j in enumerate(kfoldlist) if i != index])
    train = np.asarray(train)
    test = np.asarray(test)
    return train, test

In [12]:
def print_tree(rootnode):
    print(rootnode['gini'])
    print(rootnode['split_id'])
    print(rootnode['split_feat'])
    print(rootnode['left'])
    print("=====================================================")
    print(rootnode['right'])

In [13]:
import time
kfoldlist = np.array_split(data, 10)
accuracy = precision = recall = f_measure = 0
start = time.clock()

for i in range(len(kfoldlist)):
    train, test = split(kfoldlist,i)
    test_labels = []
    splitted = [0]*(len(data[0])-1)
    splitj=-1
    rootnode = node_to_split(train)
    rootnode = build_tree(rootnode)
    print_tree(rootnode)
    for j in range(len(test)):
        test_labels.append(classify(rootnode,test[j]))
    test_labels = np.asarray(test_labels)
    
    a, b, c, d = evaluate(test[:,-1], test_labels)
    accuracy += float(a + d)/(a + b + c + d)
    precision += float(a)/(a + c)
    recall += float(a)/(a + b)
    f_measure += float(2 * a) / ((2 * a) + b + c)
end = time.clock()


print("Time: ",end-start," secs")
print("Accuracy: ",accuracy*10,"%")
print("Precision: ",precision*10,"%")
print("Recall: ",recall*10,"%")
print("F-1 Measure: ",f_measure*10,"%")

0.1472828424545306
88
22
{'gini': 0.1161853509547341, 'split_id': 157, 'splitval': 0.1459, 'split_feat': 27, 'left': {'gini': 0.07088453721818065, 'split_id': 268, 'splitval': 17.58, 'split_feat': 20, 'left': {'gini': 0.06515805768290694, 'split_id': 31, 'splitval': 35.74, 'split_feat': 13, 'left': {'gini': 0.040250979856206404, 'split_id': 117, 'splitval': 30.25, 'split_feat': 21, 'left': {'gini': 0.006913238852402356, 'split_id': 145, 'splitval': 0.3542, 'split_feat': 25, 'left': 0, 'right': {'gini': 0.0, 'split_id': 0, 'splitval': 0.02502, 'split_feat': 15, 'left': 1, 'right': 0}}, 'right': {'gini': 0.1347881899871631, 'split_id': 12, 'splitval': 0.05271, 'split_feat': 7, 'left': {'gini': 0.08621553884711779, 'split_id': 12, 'splitval': 1.079, 'split_feat': 11, 'left': {'gini': 0.0, 'split_id': 0, 'splitval': 13.44, 'split_feat': 0, 'left': 0, 'right': 1}, 'right': {'gini': 0.0476190476190476, 'split_id': 23, 'splitval': 0.1419, 'split_feat': 24, 'left': 0, 'right': {'gini': 0.0, 's

0.13178880517029
448
22
{'gini': 0.04214167397447142, 'split_id': 206, 'splitval': 0.1607, 'split_feat': 27, 'left': {'gini': 0.02630992196209583, 'split_id': 107, 'splitval': 1.214, 'split_feat': 10, 'left': {'gini': 0.01986487396466981, 'split_id': 146, 'splitval': 0.1909, 'split_feat': 24, 'left': {'gini': 0.018365064146464043, 'split_id': 248, 'splitval': 49.11, 'split_feat': 13, 'left': {'gini': 0.012173290368779089, 'split_id': 203, 'splitval': 33.37, 'split_feat': 21, 'left': 0, 'right': {'gini': 0.07017543859649122, 'split_id': 4, 'splitval': 23.93, 'split_feat': 1, 'left': {'gini': 0.0, 'split_id': 2, 'splitval': 12.77, 'split_feat': 0, 'left': 0, 'right': 1}, 'right': 0}}, 'right': {'gini': 0.0, 'split_id': 1, 'splitval': 0.06601, 'split_feat': 5, 'left': 1, 'right': 0}}, 'right': 1}, 'right': 1}, 'right': {'gini': 0.16363636363636358, 'split_id': 1, 'splitval': 73.3, 'split_feat': 2, 'left': 0, 'right': {'gini': 0.1, 'split_id': 4, 'splitval': 0.203, 'split_feat': 8, 'left':

0.13741525101969987
248
27
{'gini': 0.07992306720388592, 'split_id': 23, 'splitval': 931.4, 'split_feat': 23, 'left': {'gini': 0.05527825249192116, 'split_id': 154, 'splitval': 102.2, 'split_feat': 22, 'left': {'gini': 0.006992834198171478, 'split_id': 101, 'splitval': 14.99, 'split_feat': 0, 'left': {'gini': 0.006628003314001652, 'split_id': 221, 'splitval': 33.37, 'split_feat': 21, 'left': 0, 'right': {'gini': 0.058823529411764705, 'split_id': 4, 'splitval': 23.93, 'split_feat': 1, 'left': {'gini': 0.0, 'split_id': 1, 'splitval': 81.72, 'split_feat': 2, 'left': 0, 'right': 1}, 'right': 0}}, 'right': 1}, 'right': {'gini': 0.21094264996704004, 'split_id': 14, 'splitval': 0.138, 'split_feat': 24, 'left': {'gini': 0.14209664209664205, 'split_id': 13, 'splitval': 0.008312, 'split_feat': 14, 'left': {'gini': 0.06060606060606061, 'split_id': 23, 'splitval': 602.9, 'split_feat': 3, 'left': {'gini': 0.0, 'split_id': 0, 'splitval': 0.1609, 'split_feat': 8, 'left': 0, 'right': 1}, 'right': 0}, 